In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
import   copy
import   xml.etree.ElementTree  as ET

# Create list for Maker’s home page URL
makers_List = ['https://www.kbb.com/acura/' , 'https://www.kbb.com/alfa-romeo/' , 'https://www.kbb.com/aston-martin/' , 'https://www.kbb.com/audi/' , 'https://www.kbb.com/bentley/',  'https://www.kbb.com/bmw/' , 'https://www.kbb.com/buick/' , 'https://www.kbb.com/cadillac/' 'https://www.kbb.com/chevrolet/' , 'https://www.kbb.com/chrysler/' , 'https://www.kbb.com/dodge/' , 'https://www.kbb.com/ferrari/' , 'https://www.kbb.com/fiat/' , 'https://www.kbb.com/ford/' , 'https://www.kbb.com/freightliner/' , 'https://www.kbb.com/genesis/' , 'https://www.kbb.com/gmc/' , 'https://www.kbb.com/honda/' , 'https://www.kbb.com/hyundai/' , 'https://www.kbb.com/infiniti/' , 'https://www.kbb.com/jaguar/' , 'https://www.kbb.com/jeep/' , 'https://www.kbb.com/kia/' , 'https://www.kbb.com/lamborghini/' , 'https://www.kbb.com/land-rover/',  'https://www.kbb.com/lexus/' , 'https://www.kbb.com/lincoln/' , 'https://www.kbb.com/maserati/' , 'https://www.kbb.com/mazda/' , 'https://www.kbb.com/mclaren/' , 'https://www.kbb.com/mercedes-benz/' , 'https://www.kbb.com/mini/' , 'https://www.kbb.com/mitsubishi/' , 'https://www.kbb.com/nissan/' , 'https://www.kbb.com/porsche/', 'https://www.kbb.com/ram/' , 'https://www.kbb.com/rivian/' , 'https://www.kbb.com/rolls-royce/' , 'https://www.kbb.com/smart/' , 'https://www.kbb.com/subaru/' , 'https://www.kbb.com/tesla/' , 'https://www.kbb.com/toyota/' , 'https://www.kbb.com/volkswagen/' , 'https://www.kbb.com/volvo/']

# Create list for annual inflation rate in the USA  (2020 - 1964)
# URL: https://www.usinflationcalculator.com/inflation/historical-inflation-rates/  
annual_inflation_rate_List = [0 , 0.018, 0.024, 0.021, 0.013, 0.001, 0.016, 0.015, 0.021, 0.032, 0.016, -0.004, 0.038, 0.028, 0.032, 0.034, 0.027, 0.023, 0.016, 0.028, 0.034, 0.022, 0.016, 0.023, 0.03, 0.028, 0.026, 0.03, 0.042, 0.054, 0.048, 0.041, 0.036, 0.019, 0.036, 0.043, 0.032, 0.062, 0.103, 0.135, 0.113, 0.076, 0.065, 0.058, 0.091, 0.11, 0.062, 0.032, 0.044, 0.057, 0.055, 0.042, 0.031, 0.029, 0.016, 0.013, 0.013]

mListSize = len(makers_List)
print ('Maker List size = ' , mListSize)
print("\n\n")
prop = {}
tmp_dct = {}
links = []
Maker = ''
first_year = ''
i = 0
m = []
values = []
model_dct_list = []                                       # list of all models for a given maker
tmp_dct_list = []                                         # list of dictionaries structure used by called functions.
model_years_dct_list = []                                 # holds detailed model info (model, first-year, price, model_url>
model_final_dct_list = []                                 # final including all years for given model

# input:  maker_url
# output: {Maker : maker, Model : v1,  First Year : v2,  New_Price : v3,  url : link}, {}….] }
def  get_maker_models(maker_url) : 
    r3 = requests.get(maker_url)
    soup1 = soup( r3.content, 'html.parser')

    # find iterative top level TAG for each maker/model
    link_as = soup1.find('div', {"class":"make-list-wrapper"}).find_all('div', {"class" : "results-row js-results-row"})
    
    i = 0
    for   y  in   link_as :                                           # y is the top-level TAG for 1st year models
        # from that top level get: links, maker, model, first-year for all top-year Models for that Maker’s new Cars
        #x = y.find('a', {"class" : "js-ymm-link result-ymm-title js.sponsored-url"}, href=True)
        x = y.find('a', href=True)
        link = "https://www.kbb.com" + x['href']                       # x['href'] has '/Maker/Model'
        if link not in links:
            links.append(link)
            maker, model = (x['href'].strip('/')).split('/')
            Maker = maker[:]
            # get the first year from "2019 Aston Marton DB11"
            ymm = x.find('h3', {"class" : "title-three"}).string             # 2019 Aston Marton DB11
            m = ymm.split()
            print("9: ", m[0], type(m[0]))                                               # m[0] is 1st year = 2019 is a string
            # first-year = m[0]
            # get the price for the first year
            price = y.find_next('span', {"class" : "title-four"}).text
            print ("price = ", price)
            L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
            L2 = [Maker, model, m[0], price, link]
            dct = dict(zip(L1, L2))
            #print (dct)
            if  i == 0 :
                model_dct_list.insert(0, dct.copy())                        # copy dct to model_dct_list[0]
            else : 
                model_dct_list.append(dct.copy())
            i = i + 1
    print ("\n\n")	 
    #print("1: ", model_dct_list) 
    return    model_dct_list                

# Input: model, fyear, fprice, model_url   1st year data for specific maker/model
# Output: [ {model1, year1, price1, model_url1}, {model1, year2, price2, model_url2}, ….]

def   get_maker_model_years_price (maker, model, f_year, f_price, model_url) :
    cur_price = 0
    next_year_price = 0
    r = requests.get(model_url)
    # create soup2 object for the model page for a specific Maker (to search recursively for <year and Link> )
    soup2 = soup( r.content, 'html.parser')
    # find years and corresponding links for this specific Maker/Model/First-Year/New_Price/Model_url of new Cars
    i = 0
    # convert price (f_price) from string ($xx,xxx) into integer binary representation
    pp = f_price.replace('$', '')
    pp = pp.replace(',', '')
    cur_price = int(pp)                                                         # convert string pp into integer
    next_year_price = cur_price                                        # integer binary representation
    
    # write/append first year data for a given model into tmp_dct_list[]
    tmp_dct = {}
    tmp_dct_list.clear()
    L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
    L2 = [maker, model, f_year, f_price, model_url]
    tmp_dct = dict(zip(L1, L2))
    tmp_dct_list.insert(0, tmp_dct.copy())              # add first year for this specific maker/model
    
    # find index in the annual_inflation_rate_list[] for this model
    if  '2020' in f_year :                                          # 2020
        i = 1                                                       # inflation starts from 2019
    elif  '2019'  in  f_year :                                      # 2019
        i = 2                                                       # inflation starts from 2018
    elif   '2018'  in f_year :                                      # 2018
        i = 3                                                       # inflation starts from 2017
    else :
        i = -1                                                      # undefined
        
    last_year = f_year[ : ]
    n = 0                                                                                      # skip first year as I have it already
    for  y  in  soup2.find_all("li", {"data-analytics" : True}) :        # remaining years
        if  n == 0 :					            # skip first year as I have it already
            n = n + 1
            continue
        mm_year = y.find_next('a', href=True)                       # mm_year = “/acura/ilx/2019”
        link = "https://www.kbb.com" + mm_year['href']
        year = y.find_next('a', href=True).contents                 # year = “2019”	
        print ("f_year, p_year", f_year, year)
        if year == last_year or year == "" :
            i = i + 1
            continue
        last_year = year[ : ]
        # estimated cur_price (binary) = int(next_year_price  / (1 + annual_inflation_rate_List[i]) )
        cur_price = int(next_year_price / (1 + annual_inflation_rate_List[i]) )	   # integer binary representation
        next_year_price = cur_price
        p1 = str(cur_price)                                         # convert binary into string
        l = len(p1) - 3                                                   # l = (string length – 3)
        p2 = p1[:l] + ',' +  p1[l:]
        p = "$" + p2                                                       # p is current price in string with right format
        #p = "N/A"
        if (len(tmp_dct)) :
            tmp_dct.clear()
        L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
        L2 = [maker, model, year, p, model_url]
        tmp_dct = dict(zip(L1, L2))         
        tmp_dct_list.append (tmp_dct.copy() )
        i = i + 1                                                                  # index into annual inflation rate array
    return  tmp_dct_list                                             # [ {<maker: <model, year, price, model_url} ]

# input: maker/model/<year : URL> and output: maker/model/years/price/{properties dictionary}
def  get_maker_model_year_price_properties(maker, model, year, price, model_year_url) :
    prop.clear()                                                                                     # clear the dictionary
    technology = []
    entertainment = []
    tmp_dct = {}
    i = 0    
    r = requests.get(model_year_url)
    # create soup3 object for the top level HTML page (to search recursively for makers, models and Links)
    soup3 = soup( r.content, 'html.parser')  
    # print(soup3.find('div', {"class":"css-1wbw7cx epomrkc1"}).find('div', {"class":"stars"}).text)
    # (1-2) find out MPG and HorsePower
    mpg = False
    hp = False
    for p  in soup3.find('div', {"class":"css-1wbw7cx epomrkc1"}).find_all('div', {"class":"stars"}) :
        i = i + 1
        #if i == 1 :
        if " MPG"  in  p.text :
            prop['MPG'] = p.text
            mpg = True
        elif " HP"  in  p.text :
            prop['HP'] = p.text
            hp = True
        else :
            continue
        
        print("i = ", i)
        
        if mpg == False :
            prop['MPG'] = "N/A"
        if hp == False :
            prop['HP'] = "N/A"
    
    #print("10: ", prop)
        #parent = str(p.find_parent('div').text).strip()
        #print("parent: ", parent)
        #if  "Combined Fuel Economy" in   parent :
            #print("parent is: combined Fuel Economy")
            #prop['MPG'] = p.text
            #print (p.text)
            #mpg = True
        #elif  "Horsepower"  in   parent :
            #print("parent is: ", "Horsepower")
            #prop['HP'] = p.text
            #hp = True
        #else :
            #break
    #if mpg == False :
        #prop['MPG'] = "N/A"
    #if hp == False :
          #prop['HP'] = "N/A"

    # (3-4) find out Expert Rating and Consumer rating
    i = 0
    for p  in  soup3.find_all('div', {"class":"css-x9skgx-NumericRating e149jidm1"}) :
        i = i + 1
        if i == 1 :
            if  p.text :
                value = p.text.replace('\xa0', '')
                prop['Expert-Rating'] = value
            else :
                prop['Expert-Rating'] = "N/A"
        elif (i == 2 ): 
            if p.text :
                value = p.text.replace('\xa0', '')
                prop['Consumer-Rating'] =  value
            else :
                prop['Consumer-Rating'] =  "N/A"
        else :
            break

    # (5) Avalon safety rating
    safety = soup3.find('div', {"class":"css-b7zitt e1pswftg4"})
    if safety :
        prop['Saftety-Rating'] = safety.contents[0]
    else :
        prop['Saftety-Rating'] = "N/A"
   
    # (6) Avalon Technology
    for  pp   in soup3.find('ul', {"class":"css-p619jy-StyledList-List"}).find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
        technology.append(pp.text)
    prop['Technology'] = technology

    # (7) Avalon Entertainment: skip Technology
    for  ppp   in   soup3.find_all('ul', {"class":"css-p619jy-StyledList-List"})[1] :
        for   e   in      ppp.find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
            entertainment.append(e.text)
 
    prop['Entertainment'] = entertainment
    print("10: prop = ", prop)
    L1 = ['Maker', 'Model', 'Year', 'New_Price', 'property']
    L2 = [maker, model, year, price, prop]
    tmp_dct = dict(zip(L1, L2))
    print ("\n\n 11: tmp_dct\n\n ", tmp_dct)
    return  tmp_dct                                                             # return prop dictionary <key : value>


# start of main program
# --------------------------------------------------------------
# select one Maker URL from the Maker list.   For the selected Maker get all corresponding models
# read list of dictionaries as the value of maker_dct[Maker].  Each is a URL for Maker page
# read one maker page Url from the makers_list[]

model_dct_list = get_maker_models(makers_List[0])
print("1: list of [model, fyear, fprice, model_url]:  " , model_dct_list)
print("\n\n")

# For all models: for each model first year , get the rest of the years data for that model
model_years_dct_list.clear()                           # clear model_final_dct[] before populating

# For each model: pass 1st year data and gets back the data for all years for that model
length  = len (model_dct_list)
print("model_dct_list length = ", length)
for    i    in  range (length) :
    tmp_dct.clear()
    tmp_dct = model_dct_list[i].copy()
    maker = tmp_dct.get('Maker')
    model = tmp_dct.get('Model')
    fyear = tmp_dct.get('Year')
    fprice = tmp_dct.get('New_Price')
    model_url = tmp_dct.get('Model-url')
    
    if i == 0 :
        #model_years_dct_list.insert(0, get_maker_model_years_price(maker, model, fyear, fprice, model_url).copy())
        model_years_dct_list[0] = copy.deepcopy(get_maker_model_years_price(maker, model, fyear, fprice, model_url))
    else :
        model_years_dct_list.append(get_maker_model_years_price(maker, model, fyear, fprice, model_url).deepcopy())
            
print("2:  model_years_dct_list[] = : ",  model_years_dct_list)
print("2: length of model_years_dct_list = ", len(model_years_dct_list))
print("done with (2) \n\n")

# for each models/years get that model/year properties into model_final_dct_list[]
model_final_dct_list.clear()                      
# clear model_final_dct[] before populating 
length = len (model_years_dct_list) 
print("8: ", length)
sum = 0
ll = len(model_years_dct_list)
for i in range (ll) :
    sum  +=  len(model_years_dct_list[i])
print("ll = \n\n", ll)
print ("sum = ", sum)
for i  in range (length):
    k = len(model_years_dct_list[i])
    tmp_dct.clear()
    tmp_dct = model_years_dct_list[i].copy()
    for j in range (k) :
        maker = tmp_dct[j].get('Maker')
        model = tmp_dct[j].get('Model')
        year = tmp_dct[j].get('Year')
        price = tmp_dct[j].get('New_Price')
        model_year_url = tmp_dct[j].get('Model-url')
        
        if  i  ==  0  and j == 0 :
            print("i = j = 0 : ", maker, model, year, price, model_year_url)
            #model_final_dct_list.insert(get_maker_model_year_price_properties(maker, model, year, price, model_year_url).copy())
            model_final_dct_list.insert(0, get_maker_model_year_price_properties(maker, model, year, price, model_year_url))
        else :
            print("i != j ", maker, model, year, price, model_year_url)
            model_final_dct_list.append(get_maker_model_year_price_properties(maker, model, year, price, model_year_url))
            
        print("\n\n12: model_final_dct_list[] = ", model_final_dct_list)

print ("3: size of model_final_dct_list = ", len(model_final_dct_list))
print("3: final model_dct_list[] = : ", model_final_dct_list)
print("\n\n")
# build the data frame for a single Maker
Maker1_df = pd.DataFrame (model_final_dct_list)
print("4: Done")

# Save Maker1_df.csv on the computer under Adam Home page\Springboard\Makers-DFs folder
#Maker1_df.to_csv (r'C:\Users\Default\Desktop\Maker1_DataFrame.csv', index=False)
Maker1_df.to_csv (r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv')
# Read the csv file back as Data Frame in your runtime Cell
# Maker1_df = pd.read_csv("C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv")

